<a href="https://colab.research.google.com/github/Yashwardhan-Ai/Chronic-Kidney-Diease-Analysis-/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install gradio

In [ ]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=7b18430ba203f0b01251cdb2c5f2d22ab7d8404537e4d6ee65636e9745aeea8b
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from fpdf import FPDF
import os
import datetime

# Load your trained model
model = load_model('/content/drive/MyDrive/kidney_model.keras')

# Define the labels for your classes
labels = ['Cyst', 'Normal', 'Stone', 'Tumor']
IMAGE_SIZE = 128

def generate_report(image, predictions_dict):
    """Generates a PDF report and returns its path."""
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=16)

    pdf.cell(200, 10, txt="Kidney Ultrasound Analysis Report", ln=1, align="C")
    pdf.cell(200, 10, txt=f"Date: {datetime.date.today().strftime('%B %d, %Y')}", ln=1, align="C")
    pdf.ln(10)

    # Save the uploaded image temporarily
    img_path = "temp_image.png"
    Image.fromarray(np.uint8(image)).save(img_path)
    pdf.image(img_path, x=10, y=pdf.get_y(), w=100)
    pdf.ln(100) # Move down after the image

    # Add prediction details
    pdf.set_font("Arial", size=12, style='B')
    pdf.cell(200, 10, txt="Prediction Details:", ln=1)
    pdf.ln(2)

    pdf.set_font("Arial", size=10)
    # The predictions passed here are raw probabilities. Multiply by 100 for the PDF.
    for label, prob in predictions_dict.items():
        pdf.cell(200, 7, txt=f"- {label}: {prob*100:.2f}%", ln=1)
    pdf.ln(10)

    # Add a note/disclaimer
    pdf.set_font("Arial", size=8)
    pdf.cell(200, 5, txt="Disclaimer: This report is for informational purposes only and should not be considered a substitute for a professional medical opinion.", ln=1)

    # Save the PDF to a file and return the path
    report_path = "kidney_analysis_report.pdf"
    pdf.output(report_path)

    # Clean up the temporary image file
    os.remove(img_path)

    return report_path


def classify_and_report(image):
    """
    Main function to classify the image, generate a report, and return both outputs for Gradio.
    """
    # Preprocess the image
    img = Image.fromarray(np.uint8(image)).resize((IMAGE_SIZE, IMAGE_SIZE))
    img_array = np.array(img).astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0) # Add a batch dimension

    # Get predictions
    predictions = model.predict(img_array)[0]

    # Normalize the predictions to ensure they sum to 1.0 (if they don't already)
    total_sum = np.sum(predictions)
    if total_sum > 0:
        normalized_predictions = predictions / total_sum
    else:
        normalized_predictions = predictions

    # Gradio requires raw probabilities (0-1) for gr.Label to work correctly.
    # The PDF generation function will handle the percentage conversion itself.
    result_dict = {labels[i]: float(normalized_predictions[i]) for i in range(len(labels))}

    # Generate the PDF report
    pdf_path = generate_report(image, result_dict)

    return result_dict, pdf_path


# Gradio Interface
interface = gr.Interface(
    fn=classify_and_report,
    inputs=gr.Image(type="numpy", label="Upload Kidney Ultrasound Image"),
    outputs=[
        gr.Label(num_top_classes=4, label="Confidence Scores"),
        gr.File(label="Download Report")
    ],
    title="Kidney Ultrasound Analysis",
    description="Upload an image to get an instant analysis and a downloadable PDF report.",
    flagging_mode='never'
)

# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6928d46a396e9246f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
